# Filtering 2



In this notebook, we cover the groupby operation which let's us summarize and filter a dataframe based on groupings of the rows that we specify.  There are three types of methods that can be used along with a group by:

- aggregate: compute statistics (mean, stdev, max, min, etc ... ) of each group.
- transformations: perform some group specific operation on the data.
- filter: filter the data based on some information form each group.

The examples in this notebook will come from Apple stock data for most of 2017.  This data is stored in AAPL.csv.  Let's first read in the data and create a column for the month and weekday.

In [2]:
import pandas as pd
import numpy as np

In [8]:
#Read in the data 
df_aapl = pd.read_csv("./Data/AAPL.csv", index_col= 0, parse_dates= ["Date"] )

#Add a month column
df_aapl["Month"] = df_aapl.Date.dt.month

#Add a week column
df_aapl["Weekday"] = df_aapl.Date.dt.day_name()
df_aapl.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Month,Weekday
0,2017-01-03,115.800003,116.330002,114.760002,116.150002,114.311760,28781900,1,Tuesday
1,2017-01-04,115.849998,116.510002,115.750000,116.019997,114.183815,21118100,1,Wednesday
2,2017-01-05,115.919998,116.860001,115.809998,116.610001,114.764473,22193600,1,Thursday
3,2017-01-06,116.779999,118.160004,116.470001,117.910004,116.043915,31751900,1,Friday
4,2017-01-09,117.949997,119.430000,117.940002,118.989998,117.106812,33561900,1,Monday


In [3]:
#shape of df
df_aapl.shape

(169, 9)

In [5]:
#How columns are stored
df_aapl.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
Month                 int64
Weekday              object
dtype: object

Let's say I want to the know the average open price for each month.  I will groupby month and for each of these groups I want to the know the average open price.

In [6]:
#Basic groupby
df_aapl.groupby(by = "Month").Open.mean()

Month
1    119.093499
2    133.234738
3    140.362174
4    143.030001
5    151.965908
6    148.215001
7    148.096500
8    158.946958
9    164.800003
Name: Open, dtype: float64

As you see, I get back a series with the avg open price for each month.  

What is I want to know both the average open price and the average volume in each month.  To do this, I just select the two column  columns instead of just the Open column.

In [7]:
#Summarize groups based on two columns
df_aapl.groupby(by = "Month")["Open", "Volume"].mean()

,Open,Volume
Month,,
1,119.093499,2.815610e+07
2,133.234738,3.026151e+07
3,140.362174,2.441863e+07
4,143.030001,1.964758e+07
5,151.965908,2.971615e+07
6,148.215001,3.109900e+07
7,148.096500,2.109962e+07
8,158.946958,2.874213e+07
9,164.800003,1.659110e+07


In this case, I get back a data frame with the information from the each month given in the two columns.  

What if I want the average open price for each month and each day of the week? I can specify a list of column names for the by argument in the group by to accomplish this.

In [8]:
#Groubpy multiple columns
df_aapl.groupby(by = ["Month", "Weekday"]).Open.mean().head(15)

Month  Weekday  
1      Friday       119.619999
       Monday       119.626666
       Thursday     118.972500
       Tuesday      118.722000
       Wednesday    118.752499
2      Friday       132.945004
       Monday       133.116669
       Thursday     133.170000
       Tuesday      134.329998
       Wednesday    132.582500
3      Friday       140.850000
       Monday       139.502499
       Thursday     140.982001
       Tuesday      140.345001
       Wednesday    139.956000
Name: Open, dtype: float64

I now have a group for every month + weekday combination.  The above is a series with multi-level.  In the PPT I show how to slice the series in this case.  Another way to get around this is to simply reset the index, which will move the multilevel index to columns in your dataframe.

In [9]:
df_aapl.groupby(by = ["Month", "Weekday"]).Open.mean().reset_index().head(15)

,Month,Weekday,Open
0,1,Friday,119.619999
1,1,Monday,119.626666
2,1,Thursday,118.972500
3,1,Tuesday,118.722000
4,1,Wednesday,118.752499
5,2,Friday,132.945004
6,2,Monday,133.116669
7,2,Thursday,133.170000
8,2,Tuesday,134.329998
9,2,Wednesday,132.582500


## Aggregate

I can use the aggregate method to summarize the groups using multiple functions. Let's say I want to know the avg and standard deviation of the open price for each month. After the groupby, I use the agg() method and inside I can specify a list of functions that will be used to summarize each group.

In [10]:
#Using agg
df_aapl.groupby(by = "Month").Open.agg([np.mean, np.std])

,mean,std
Month,,
1,119.093499,1.891817
2,133.234738,3.410836
3,140.362174,1.728701
4,143.030001,1.120391
5,151.965908,3.476493
6,148.215001,4.450870
7,148.096500,3.479140
8,158.946958,2.926976
9,164.800003,NaN


Let's now use agg() where we select both Open and Volume:

In [11]:
#Using agg() with
result  = df_aapl.groupby(by = "Month")["Open", "Volume"].agg([np.mean, np.std])

result

Open                  Volume              
             mean       std          mean           std
Month                                                  
1      119.093499  1.891817  2.815610e+07  6.572536e+06
2      133.234738  3.410836  3.026151e+07  2.055055e+07
3      140.362174  1.728701  2.441863e+07  7.416385e+06
4      143.030001  1.120391  1.964758e+07  4.026376e+06
5      151.965908  3.476493  2.971615e+07  1.014266e+07
6      148.215001  4.450870  3.109900e+07  1.416738e+07
7      148.096500  3.479140  2.109962e+07  4.182316e+06
8      158.946958  2.926976  2.874213e+07  1.037540e+07
9      164.800003       NaN  1.659110e+07           NaN

This gives back a dataframe with multi-level columns, which can be sliced as follows:

In [12]:
#Get the columns for the open price
result.loc[:, "Open"]

,mean,std
Month,,
1,119.093499,1.891817
2,133.234738,3.410836
3,140.362174,1.728701
4,143.030001,1.120391
5,151.965908,3.476493
6,148.215001,4.450870
7,148.096500,3.479140
8,158.946958,2.926976
9,164.800003,NaN


Let's say that instead of applying both functions to both columns I wanted the mean to be applied to the Open column and a maximum to be applied to the volume column.  To do this, I specify a dictionary in the agg() method where the key is the column and the value is a function to be applied to the groups of that column. In this case, since I specify the columns I want to focus on in the dictionary I do not have to select them after the groupby

In [13]:
#agg with a dictionary
df_aapl.groupby(by = "Month").agg({"Open": np.mean,"Volume": np.max})

,Open,Volume
Month,,
1,119.093499,49201000
2,133.234738,111985000
3,140.362174,43885000
4,143.030001,30379400
5,151.965908,50767700
6,148.215001,72307300
7,148.096500,32476300
8,158.946958,69936800
9,164.800003,16591100


I can even use the agg() with a customer function.  Let's say I wanted the count of the number of days in each month the open price was above the average open price for the month. First I'll write my customer function to be applied to each group.

In [15]:
def Compare_Open(group):
    count = 0
    
    avg_price = group.mean()
    for index in list(group.index):
        open_price = group[index]
        
        if open_price>=avg_price:
            
            count+=1
            
    return count
        

In [16]:
df_aapl.groupby(by = "Month").agg({"Open":Compare_Open})

,Open
Month,
1,11.0
2,10.0
3,10.0
4,11.0
5,15.0
6,7.0
7,11.0
8,14.0
9,1.0


In the example above, the function Compare_Open is called once for each group (each month),  The input group is a series that represents the open price for the given group.

## Transform

Instead of aggregating each group we can apply a transformation with transform() method after the groupby.  The agg() method uses takes a column of data and spits out a single number summarizing this column based on the specified groups.  The transform() method takes a column and returns a back a series that is the same length. For example let's say I use the transform method with the sum function on  the column Volume grouping by Month.

In [49]:
volume_by_month = df_aapl.groupby(by = "Month").Volume.transform(np.sum)

volume_by_month.head()

0    563122000
1    563122000
2    563122000
3    563122000
4    563122000
Name: Volume, dtype: int64

In [50]:
#Look at how many rows there ar
volume_by_month.shape

(169,)

Notice that this series has 169 rows, which is the same number of days that we have stock information for.  What happened is that for each group we compute the sum, but instead of giving a single number for each group, it takes this summarizing number of matches it with the group that each row corresponds to.  So let's say I wanted a column for the fraction of the month's volumne that each day represents.  I can first use a transform as I have done above to get a column for the total volume in each month and then I can do the simple division to get the desired column.

In [55]:
#Use transform to create a column for the total volumn in each month
df_aapl["Total_Month_Volume"] = df_aapl.groupby(by = "Month").Volume.transform(np.sum)

df_aapl.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Month,Weekday,Total_Month_Volumn,Total_Month_Volume
0,2017-01-03,115.800003,116.330002,114.760002,116.150002,114.311760,28781900,1,Tuesday,563122000,563122000
1,2017-01-04,115.849998,116.510002,115.750000,116.019997,114.183815,21118100,1,Wednesday,563122000,563122000
2,2017-01-05,115.919998,116.860001,115.809998,116.610001,114.764473,22193600,1,Thursday,563122000,563122000
3,2017-01-06,116.779999,118.160004,116.470001,117.910004,116.043915,31751900,1,Friday,563122000,563122000
4,2017-01-09,117.949997,119.430000,117.940002,118.989998,117.106812,33561900,1,Monday,563122000,563122000


In [56]:
#Create new column for the fraction of the month's volumen
df_aapl["Frac_Months_Vol"] = df_aapl["Volume"]/df_aapl["Total_Month_Volume"]

df_aapl.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Month,Weekday,Total_Month_Volumn,Total_Month_Volume,Frac_Months_Vol
0,2017-01-03,115.800003,116.330002,114.760002,116.150002,114.311760,28781900,1,Tuesday,563122000,563122000,0.051111
1,2017-01-04,115.849998,116.510002,115.750000,116.019997,114.183815,21118100,1,Wednesday,563122000,563122000,0.037502
2,2017-01-05,115.919998,116.860001,115.809998,116.610001,114.764473,22193600,1,Thursday,563122000,563122000,0.039412
3,2017-01-06,116.779999,118.160004,116.470001,117.910004,116.043915,31751900,1,Friday,563122000,563122000,0.056385
4,2017-01-09,117.949997,119.430000,117.940002,118.989998,117.106812,33561900,1,Monday,563122000,563122000,0.059600


I can also use transform to create standardize columns for each group.  So let's say I want to create a standardized open price column where for each open price I substract off the mean open price for the month and divide by the standard deviation.  We can use a lambda function to accomplish this.

In [57]:
#Using the tranform method to standardize
df_aapl["Standard_Open"] = df_aapl.groupby(by = "Month").Open.transform(lambda x : (x - x.mean())/x.std())

df_aapl.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Month,Weekday,Total_Month_Volumn,Total_Month_Volume,Frac_Months_Vol,Standard_Open
0,2017-01-03,115.800003,116.330002,114.760002,116.150002,114.311760,28781900,1,Tuesday,563122000,563122000,0.051111,-1.740917
1,2017-01-04,115.849998,116.510002,115.750000,116.019997,114.183815,21118100,1,Wednesday,563122000,563122000,0.037502,-1.714490
2,2017-01-05,115.919998,116.860001,115.809998,116.610001,114.764473,22193600,1,Thursday,563122000,563122000,0.039412,-1.677489
3,2017-01-06,116.779999,118.160004,116.470001,117.910004,116.043915,31751900,1,Friday,563122000,563122000,0.056385,-1.222899
4,2017-01-09,117.949997,119.430000,117.940002,118.989998,117.106812,33561900,1,Monday,563122000,563122000,0.059600,-0.604447


The lambda function above is only called only once for each group where the input x will be a series representing the column Open for each of the Months that we grouped by.  It should be noted that if you subtract a single number from a series, as is the case when this lambda function is called, then pandas knows to subtract this number from all the numbers in the series.  The same game for dividing a series by a single number.

## Filtering

The filter() method after a group by let's us only select rows corresponding to each group where a certain criterion regarding the group as a whole is true is satisfied.  The filter() method must take in a function that returns a boolean.  The function will be run once for each row. For example, lets say I only want to look at rows for days in months where the average opening price for the month was above 140.

In [62]:
above_140 = df_aapl.groupby(by = "Month").filter(lambda group: group.Open.mean() >= 140)

#Note that we lose Jan and Feb since these months had an avg open below 140.
above_140.Month.unique()

array([3, 4, 5, 6, 7, 8, 9])

The lambda function above is called once for each group where the input group will be dataframe with the rows corresponding to the given month that is being run through the lambda function.